In [1]:
from glob import glob
from streaming import MDSWriter
from streaming import LocalDataset
from transformers import default_data_collator, DataCollatorForLanguageModeling
from tqdm import tqdm
import numpy as np

In [2]:
folders = sorted(glob('tokenized_indexes/tokenized-*'), key = lambda x: int(x.split('-')[-1]))

In [3]:
folders.extend(sorted(glob('tokenized_extra/tokenized-*'), key = lambda x: int(x.split('-')[-1])))

In [4]:
folders

['tokenized_indexes/tokenized-0',
 'tokenized_indexes/tokenized-1',
 'tokenized_indexes/tokenized-2',
 'tokenized_indexes/tokenized-3',
 'tokenized_indexes/tokenized-4',
 'tokenized_indexes/tokenized-5',
 'tokenized_indexes/tokenized-6',
 'tokenized_indexes/tokenized-7',
 'tokenized_indexes/tokenized-8',
 'tokenized_indexes/tokenized-9',
 'tokenized_indexes/tokenized-10',
 'tokenized_indexes/tokenized-11',
 'tokenized_indexes/tokenized-12',
 'tokenized_extra/tokenized-0',
 'tokenized_extra/tokenized-1',
 'tokenized_extra/tokenized-2',
 'tokenized_extra/tokenized-3',
 'tokenized_extra/tokenized-4',
 'tokenized_extra/tokenized-5',
 'tokenized_extra/tokenized-6',
 'tokenized_extra/tokenized-7',
 'tokenized_extra/tokenized-8',
 'tokenized_extra/tokenized-9',
 'tokenized_extra/tokenized-10',
 'tokenized_extra/tokenized-11',
 'tokenized_extra/tokenized-12']

In [4]:
from streaming.base.format.mds.encodings import Encoding, _encodings

class UInt16(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint16)

_encodings['uint16'] = UInt16

In [5]:
columns = {
    'input_ids': 'uint16',
}

compression = 'zstd'
hashes = 'sha1', 'xxh64'

In [6]:
!rm -rf combine-all

In [7]:
dataset = LocalDataset(local=folders[0])

In [8]:
with MDSWriter(out='combine-all', columns=columns, compression=None, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = LocalDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

100%|██████████| 20637/20637 [00:00<00:00, 22289.75it/s]


In [12]:
dataset = LocalDataset(local='combine-all')

In [13]:
len(dataset) * 4096

14114934784